In [1]:
import os
from tools.procOps import *
from tools.fileOps import *
import pandas as pd
from cat.hints_db import *
from tools.misc import *
from tools.intervals import *
from tools.transcripts import *
import itertools
import re
from cat.hgm import extract_exon_hints

In [2]:
# to run homGeneMapping, I need to use the existing hints database which contains the IsoSeq hints

# for each genome, I construct a reference GTF consisting of ab-initio exons and CDS
# I then lift them over to target genomes

# we want to use both CDS and exon intervals because CGP has a coding-only model, and so the CDS interval
# will be the exonic interval

# I also construct a supplemental GFF that consists of the transMap exons (or gencode for human)
# these will let me know which exons are novel

In [5]:
gencode_gp = '../../reference/gencode.v27.annotation.no_PAR.gp'

def gene_pred_to_gtf(tx):
    """Extract both exons and CDS intervals as CDS"""
    for exon in tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'transcript_id "{}; gene_id "{}"'.format(tx.name, tx.name2)]
    cds_tx = Transcript(tx.get_bed(new_start=tx.thick_start, new_stop=tx.thick_stop))
    for exon in cds_tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'transcript_id "{}_cds"; gene_id "{}"'.format(tx.name, tx.name2)]
        
def gene_pred_to_gff(tx):
    """Extract both exons and CDS intervals as CDS"""
    for exon in tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'source=A']
    cds_tx = Transcript(tx.get_bed(new_start=tx.thick_start, new_stop=tx.thick_stop))
    for exon in cds_tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'source=A']
        

with open('gtfs.tbl', 'w') as outf:
    for genome in ['Human', 'Clint_Chimp', 'Susie_Gorilla', 'Susie_Orangutan']:
        # isolate ab-initio transcripts that are spliced
        cgp_gp = '../../augustus_cgp/{}.augCGP.gp'.format(genome)
        pb_gp = '../../augustus_pb/{}.augPB.gp'.format(genome)
        out_gtf = genome + '.ab_initio_exons.gtf'
        with open(out_gtf, 'w') as out_gtf_handle:
            for gp in [cgp_gp, pb_gp]:
                for tx in gene_pred_iterator(gp):
                    if len(tx.intron_intervals) > 0:
                        print_rows(out_gtf_handle, list(gene_pred_to_gtf(tx)))
        supp_gff = genome + '.extrinsic_supplement.gff'
        with open(supp_gff, 'w') as supp_gff_handle:
            # copy all transMap or GENCODE exons with source=A to the supplement
            if genome == 'Human':
                annot_gp = gencode_gp
            else:
                annot_gp = '../../transMap/{}.gp'.format(genome)
            for tx in gene_pred_iterator(annot_gp):
                print_rows(supp_gff_handle, list(gene_pred_to_gff(tx)))
        print_row(outf, [genome, out_gtf, supp_gff])

In [6]:
!homGeneMapping --halfile=../../v2_chimp_orang.hal --gtfs=gtfs.tbl --cpus=32 --dbaccess=../../hints_database/hints.db

halLiftover starts. Processing
Clint_Chimp
Human
Susie_Gorilla
Susie_Orangutan


In [7]:
# load HGM output
parsed_hgm = {}
r = re.compile('\**-')
species_map = {'0': 'Clint_Chimp', '1': 'Human', '2': 'Susie_Gorilla', '3': 'Susie_Orangutan'}

def parse_info_line(hgm_info):
    """
    For a given info line, we want to know what genomes it was supported in, and at what level
    
    Returns a dict with key-values of PB/N/M, etc, value is multiplicity
    """
    ret_dict = {}  # keeps mapping of genome name to values
    for l in hgm_info.split(','):
        ret = {}
        if 'PB' in l or 'A' in l:  # make sure we have something to look at
            g = species_map[l[0]]
            sl = l[1:].split(':')  # strip species identifier
            for v in sl:
                if '-' in v:
                    feature_type, mult = r.split(v)
                    ret[feature_type] = int(mult)
                else:
                    ret[v] = 1
            ret_dict[g] = ret
    return ret_dict

for g in ['Human', 'Clint_Chimp', 'Susie_Gorilla', 'Susie_Orangutan']:
    e = []
    with open(g + '.gtf') as infile:
        for line in infile:
            if '\texon\t' in line:
                line = line.rstrip().split('\t')
                attr_line = line[-1]
                attributes = parse_gtf_attr_line(attr_line)
                data = parse_info_line(attributes['hgm_info'])
                i = ChromosomeInterval(line[0], int(line[3]) - 1, int(line[4]), '.', data)
                e.append([i, attributes['transcript_id'].split('_')[0]])
    parsed_hgm[g] = e

In [8]:
supported_dfs = {}
features = ['PB', 'A']
cols = ['ab_initio_tx_id', 'chromosome', 'start', 'stop']
for g in ['Human', 'Clint_Chimp', 'Susie_Gorilla', 'Susie_Orangutan']:
    for m in features:
        cols.append(g + '_' + m)

for genome, e in parsed_hgm.iteritems():
    df = []  # dataframe with values chromosome, start, stop, then genome support levels in order
    for i, tx_id in e:
        # filter out poorly annotated human sequences
        if genome == 'Human' and ('alt' in i.chromosome or 'random' in i.chromosome or 'chrUn' in i.chromosome):
            continue
        r = [tx_id, i.chromosome, i.start, i.stop]  # row
        for g in ['Human', 'Clint_Chimp', 'Susie_Gorilla', 'Susie_Orangutan']:
            for m in features:
                if g in i.data:
                    r.append(i.data[g].get(m, 0))
                else:
                    r.append(0)
        df.append(r)
    
    df = pd.DataFrame(df, columns=cols)
    supported_dfs[genome] = df.drop_duplicates()
            

In [14]:
# load gene info for output
genomes = ['Clint_Chimp', 'Susie_Gorilla', 'Susie_Orangutan']
annotation_maps = {}
for genome in genomes:
    df = pd.read_csv(os.path.join('../../consensus_gene_set', genome + '.gp_info'), sep='\t')
    df = df[['alignment_id', 'source_gene', 'source_gene_common_name']]
    df.columns = ['ab_initio_tx_id', 'gene_id', 'gene_name']
    annotation_maps[genome] = df

In [11]:
# perform parent gene assignment on Human
from cat.parent_gene_assignment import assign_parents
parents = []
for d in ['../../augustus_cgp/Human.augCGP.gp', '../../augustus_pb/Human.augPB.gp']:
    tx_dict = get_gene_pred_dict(d)
    tx_dict = {tx_id: tx for tx_id, tx in tx_dict.iteritems() if len(tx.exon_intervals) > 1}
    # abuse parental assignment code by passing reference as unfiltered and filtered transMap
    parents.append(assign_parents('../../reference/gencode.v27.annotation.no_PAR.gp', 
                                  '../../reference/gencode.v27.annotation.no_PAR.gp', 
                                  '../../genome_files/Human.chrom.sizes', d))
assignment_df = pd.concat(parents)

In [12]:
from tools.sqlInterface import *
ref_df = load_annotation('../../databases/Human.db')

In [18]:
human_df = assignment_df.reset_index().merge(ref_df, left_on='AssignedGeneId', right_on='GeneId')
human_df = human_df[['TranscriptId_x', 'GeneId', 'GeneName']]
human_df.columns = ['ab_initio_tx_id', 'gene_id', 'gene_name']
annotation_maps['Human'] = human_df

In [27]:
strict_novel_exon_dfs = {}
for g, df in supported_dfs.iteritems():
    subset_df = df[(df['{}_PB'.format(g)] > 0) & (df['Human_A'] == 0) & (df['Clint_Chimp_A'] == 0) & (df['Susie_Gorilla_A'] == 0) & (df['Susie_Orangutan_A'] == 0)]
    subset_df = subset_df.merge(annotation_maps[g], on='ab_initio_tx_id')
    subset_df = subset_df.drop(['Clint_Chimp_A', 'Human_A', 'Susie_Gorilla_A', 'Susie_Orangutan_A'], axis=1)
    # drop duplicates
    subset_df = subset_df.groupby(['chromosome', 'start', 'stop']).first().reset_index()
    subset_df.to_csv('{}.txt'.format(g), sep='\t', index=False)
    strict_novel_exon_dfs[g] = subset_df

In [28]:
# for each, how many have support in others?
# unroll

%matplotlib inline
from matplotlib_venn import *
import matplotlib.pyplot as plt
genomes = ['Human', 'Clint_Chimp', 'Susie_Gorilla', 'Susie_Orangutan']
def unroll_df(df, g):
    other_genomes = set(genomes) - {g}
    results = defaultdict(set)
    singletons = 0
    for i, s in df.iterrows():
        if all(s['{}_PB'.format(og)] == 0 for og in other_genomes):
            singletons += 1
            continue
        for og in other_genomes:
            if s['{}_PB'.format(og)] > 0:
                results[og].add(i)
    return results, singletons

from cat.plots import *
with open('novel_exon_cross_species_support.pdf', 'w') as outf, PdfPages(outf) as pdf:
    for g, df in strict_novel_exon_dfs.iteritems():
        fig, ax = plt.subplots()
        results, singletons = unroll_df(df, g)
        x = sorted(results.items())
        names, vals = zip(*x)
        v = venn3(vals, set_labels=names, ax=ax)
        ax.set_title('{} novel exons supported by IsoSeq in other genomes\nwith {:,} species-specific exons'.format(g, singletons))
        multipage_close(pdf, False)

In [24]:
# how many of the strict set have no overlap with any comparative exon?
for genome, df in strict_novel_exon_dfs.iteritems():
    df[['chromosome', 'start', 'stop']].to_csv('{}.bed'.format(genome), sep='\t', header=None, index=False)
    with TemporaryFilePath() as tmp_annotation_bed, TemporaryFilePath() as tmp_annotation_bed_flat:
        if genome == 'Human':
            !cp '../../reference/gencode.v27.annotation.no_PAR.bed' {tmp_annotation_bed}
        else:
            !genePredToBed ../../transMap/{genome}.gp {tmp_annotation_bed}
        # flatten annotation to split apart exons for intersection
        !bedtools bed12tobed6 -i {tmp_annotation_bed} > {tmp_annotation_bed_flat}
        !bedSort {tmp_annotation_bed_flat} {tmp_annotation_bed_flat}
        # find complement of intersection of flat per-exon BED with novel calls
        !bedtools intersect -v -a {genome}.bed -b {tmp_annotation_bed_flat} > {genome}.no_overlap.txt
        # find intersection of the above with extant loci (how many novel exons are within an annotated gene)
        !bedtools intersect -u -a {genome}.no_overlap.txt -b {tmp_annotation_bed} > {genome}.no_overlap.extant_only.txt
        # re-add columns
        for x in ['.no_overlap.extant_only.txt', '.no_overlap.txt']:
            p = genome + x
            tmp_df = pd.read_csv(p, sep='\t', header=None)
            tmp_df.columns = ['chromosome', 'start', 'stop']
            tmp_df = tmp_df.merge(df, on=['chromosome', 'start', 'stop'])
            tmp_df.to_csv(p, sep='\t', index=False)

In [31]:
from cat.plots import *
with open('novel_exon_cross_species_support_no_splice_shift.pdf', 'w') as outf, PdfPages(outf) as pdf:
    for g in strict_novel_exon_dfs:
        if g == 'Human':
            continue
        fig, (ax1, ax2) = plt.subplots(ncols=2)
        for x, ax in zip(*[['.no_overlap.extant_only.txt', '.no_overlap.txt'], [ax1, ax2]]):
            p = g + x
            df = pd.read_csv(p, sep='\t')
            results, singletons = unroll_df(df, g)
            d = sorted(results.items())
            names, vals = zip(*d)
            v = venn3(vals, set_labels=names, ax=ax)
            if x == '.no_overlap.extant_only.txt':
                ax.set_title('Novel exons in annotated loci\n{:,} species-specific exons'.format(singletons))
            else:
                ax.set_title('All novel exons\n{:,} species-specific exons'.format(singletons))
        fig.suptitle(g)
        multipage_close(pdf, False)

In [36]:
rdf = []
for g in strict_novel_exon_dfs:
    tot = len(open(g + '.txt').readlines())
    exon = len(open(g + '.no_overlap.txt').readlines())
    loci = len(open(g + '.no_overlap.extant_only.txt').readlines())
    rdf.append([g, tot - (exon + loci), exon - loci, loci])
    
rdf = pd.DataFrame(rdf, columns=['genome', 'Novel splices', 'Novel exons', 'Novel exons in previously annotated loci'])
print rdf.sort_values('genome')

            genome  Novel splices  Novel exons  \
0      Clint_Chimp            477          305   
2            Human             96           50   
3    Susie_Gorilla             81          248   
1  Susie_Orangutan            407          229   

   Novel exons in previously annotated loci  
0                                       173  
2                                         7  
3                                        18  
1                                       164  


In [56]:
# Evan wants me to add information on how many amino acids these add.

# I also realized I am double counting events

ref_tmp = ref_df[['GeneId', 'GeneBiotype']]
ref_tmp.columns = ['gene_id', 'gene_biotype']
ref_tmp = ref_tmp.drop_duplicates()
# just load from the disk

novel_dfs = {}
for g in strict_novel_exon_dfs:
    splices = pd.read_csv(g + '.txt', sep='\t')
    exon = pd.read_csv(g + '.no_overlap.txt', sep='\t')
    # remove exon from splices
    tmp = splices.merge(exon, how='outer', indicator=True)
    splices = tmp[tmp['_merge'] == 'left_only'].drop('_merge', axis=1).merge(ref_tmp, on='gene_id')
    exon = exon.merge(ref_tmp, on='gene_id')
    novel_dfs[g] = [splices, exon]
    print g, len(splices), len(exon)

Clint_Chimp 624 174 Counter({u'protein_coding': 600, u'lincRNA': 7, u'antisense_RNA': 5, u'processed_transcript': 5, u'transcribed_unprocessed_pseudogene': 3, u'unprocessed_pseudogene': 1, u'sense_overlapping': 1, u'transcribed_processed_pseudogene': 1, u'transcribed_unitary_pseudogene': 1})
Susie_Orangutan 545 139 Counter({u'protein_coding': 511, u'unprocessed_pseudogene': 13, u'antisense_RNA': 6, u'transcribed_unprocessed_pseudogene': 6, u'processed_transcript': 5, u'lincRNA': 2, u'transcribed_processed_pseudogene': 1, u'processed_pseudogene': 1})
Human 103 56 Counter({u'protein_coding': 101, u'transcribed_processed_pseudogene': 1, u'lincRNA': 1})
Susie_Gorilla 84 68 Counter({u'protein_coding': 75, u'unprocessed_pseudogene': 3, u'sense_overlapping': 3, u'lincRNA': 1, u'transcribed_unprocessed_pseudogene': 1, u'transcribed_unitary_pseudogene': 1})


In [ ]:
tx_dicts = {}
for g in novel_dfs.iteritems():
    if g == 'Human':
        tx_dicts[g] = get_gene_pred_dict('/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/reference/gencode.v27.annotation.no_PAR.gp')
    else:
        tx_dicts[g] = get_gene_pred_dict('/hive/groups/recon/projs/primates/primates_indel_corrected_bionano_cut/consensus_gene_set/{}.gp'.format(g))
    

In [ ]:
for g, (splices, exon) in novel_dfs.iteritems():
    